In [2]:
%matplotlib inline
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import theano.tensor as T
import theano
import lasagne
import lasagne.nonlinearities as nonlin
import pylab
import pickle
import re
import os
import imageio
import time
from IPython.display import display, clear_output
import PIL

from net.pretrained.vgg16.file_worker import make_network
from other.preprocess_img import preprocess, resize_img

In [7]:
dataDir='data/annotations_trainval2017'
dataType='val2017'
annFile='{}/annotations/captions_{}.json'.format(dataDir, dataType)

# img2vec

In [8]:
def imgs_load(dir_name):
    imgs = {}
    for file_name in os.listdir(dir_name + '/data'):
        if '.' in file_name and file_name.split('.')[1] == 'data':
            with open('{}/data/{}'.format(dir_name, file_name), 'rb') as fl:
                data = pickle.load(fl)
                imgs.update(data)
    return imgs

In [9]:
imgs = imgs_load(dataDir)
display(len(imgs))

4991

# texts

In [11]:
from other.sents_process import *

In [16]:
danno = sents_load(dataDir)

In [15]:
danno = {}
for id, v in anno:
    danno[id] = danno.get(id, []) + [v]

ValueError: too many values to unpack (expected 2)

# neural networks

In [17]:
from net.train import *

# CNN

Generate X, y

In [18]:
H = 300
W = 14

In [19]:
train_size = 23300

X, y = [], []
for id, data in imgs.items():
    id = id.lstrip('0')
    try:
        for ls in danno[id]:
            vects = []
            for elem in ls:
                if not isinstance(elem, str):
                    vects.append(elem)
            vects = np.pad(vects, [(0, W), (0, 0)], mode='constant', constant_values=0)[:W].T
            X.append([vects])
            y.append(data[1])
    except:
        pass

print(len(X))

X_train, y_train, X_val, y_val = X[:train_size], y[:train_size], X[train_size:], y[train_size:]

23968


# new version

In [20]:
def build_cnn(file_name=None):
    net = NeuralNet(must_have=[
            'input_shape', 
            'learning_rate', 
            'train_fun', 
            'loss_fun', 
            'loss_fun_det', 
            'predict_fun_det'])
    
    input_shape = [None, H, W]
    
    input_X = T.tensor3("input X", dtype='float32')
    target_y = T.matrix("target Y", dtype='float32')

    
    net['inp'] = lasagne.layers.InputLayer(input_shape, input_var=input_X)
    
    net['max'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.max)
    net['min'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.min)
    net['mean'] = lasagne.layers.GlobalPoolLayer(net['inp'], pool_function=theano.tensor.mean)
    
    net['con_2'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=2,nonlinearity=None)
    net['con_3'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=3,nonlinearity=None)
    net['con_4'] = lasagne.layers.Conv1DLayer(net['inp'], num_filters=64, filter_size=4,nonlinearity=None)
    
    boltzmann_max = lambda a, axis: T.sum(a * T.exp(a), axis=-1) / T.exp(a).sum(-1)
    
    net['gmax_2b'] = lasagne.layers.GlobalPoolLayer(net['con_2'], pool_function=boltzmann_max)
    net['gmax_3b'] = lasagne.layers.GlobalPoolLayer(net['con_3'], pool_function=boltzmann_max)
    net['gmax_4b'] = lasagne.layers.GlobalPoolLayer(net['con_4'], pool_function=boltzmann_max)
    
    net['merge'] = lasagne.layers.ConcatLayer((net['max'], net['min'], net['mean'], 
                                               net['gmax_2b'], net['gmax_3b'], net['gmax_4b']))
    
    net['batch_0'] =  lasagne.layers.batch_norm(net['merge'])
    
    net['dens_1'] = lasagne.layers.DenseLayer(net['batch_0'], num_units=1000, nonlinearity=nonlin.elu)
    net['batch_1'] =  lasagne.layers.batch_norm(net['dens_1'])
    net['drop_1'] = lasagne.layers.DropoutLayer(net['batch_1'], p=0.6)
    
    net['dens_2'] = lasagne.layers.DenseLayer(net['drop_1'], num_units=1000, nonlinearity=nonlin.elu)
    net['batch_2'] =  lasagne.layers.batch_norm(net['dens_2'])
    net['drop_2'] = lasagne.layers.DropoutLayer(net['batch_2'], p=0.6)
    
    net['last'] = lasagne.layers.DenseLayer(net['drop_2'], num_units=4096)
    
    
    y_predicted = lasagne.layers.get_output(net['last'])
    y_predicted_det = lasagne.layers.get_output(net['last'], deterministic=True)

    all_weights = lasagne.layers.get_all_params(net['last'], trainable=True)
    
    learning_rate = theano.shared(lasagne.utils.floatX(0.001))
    loss = lasagne.objectives.squared_error(target_y, y_predicted).mean()
    loss_det = lasagne.objectives.squared_error(target_y, y_predicted_det).mean()
    
    # loss = loss + lasagne.regularization.regularize_layer_params(net['last'], lasagne.regularization.l2) * 0.01
    updates = lasagne.updates.adam(loss, all_weights, learning_rate=learning_rate)
    
    train_fun = theano.function([input_X, target_y], loss, updates=updates, allow_input_downcast=True)
    loss_fun = theano.function([input_X, target_y], loss, allow_input_downcast=True)
    loss_fun_det = theano.function([input_X, target_y], loss_det, allow_input_downcast=True)
    predict_fun_det = theano.function([input_X], y_predicted_det, allow_input_downcast=True)
    
    if file_name:
        load_net(net['last'], file_name, dataDir)
        
    def proc_vec(v):
        res = []
        for vv in v:
            res.append(vv[0])
        return res
    
    def proc_1(fun):
        def temp(v):
            v = proc_vec(v)
            return fun(v)
        return temp

    def proc_2(fun):
        def temp(v, u):
            v = proc_vec(v)
            return fun(v, u)

        return temp

    net.input_shape = input_shape
    net.learning_rate = learning_rate
    net.train_fun = proc_2(train_fun)
    net.loss_fun = proc_2(loss_fun)
    net.loss_fun_det = proc_2(loss_fun_det)
    net.predict_fun_det = proc_1(predict_fun_det)
    
    return net.check()

In [21]:
net = build_cnn()

NameError: name 'NeuralNet' is not defined

In [18]:
for key,val in net.items():
    if isinstance(val,lasagne.layers.Layer):
        params = val.get_params()
        print (key, val.output_shape, [p.get_value().shape for p in params])

gmax_3b (None, 64) []
dens_2 (None, 1000) [(1000, 1000)]
con_3 (None, 64, 12) [(64, 300, 3), (64,)]
inp (None, 300, 14) []
gmax_2b (None, 64) []
gmax_4b (None, 64) []
con_4 (None, 64, 11) [(64, 300, 4), (64,)]
batch_1 (None, 1000) []
drop_1 (None, 1000) []
merge (None, 1092) []
min (None, 300) []
batch_0 (None, 1092) [(1092,), (1092,), (1092,), (1092,)]
mean (None, 300) []
last (None, 4096) [(1000, 4096), (4096,)]
dens_1 (None, 1000) [(1092, 1000)]
con_2 (None, 64, 13) [(64, 300, 2), (64,)]
max (None, 300) []
batch_2 (None, 1000) []
drop_2 (None, 1000) []


In [19]:
training_c = Training(net, 40)
training_c.set_Xy(X_train, y_train, X_val, y_val)

In [20]:
from pandas import ewma

def my_plot(losses_val, losses_train):
    plt.figure(figsize=(16, 8))
    
    losses_train = ewma(np.array(losses_train), span=70)
    losses_val = ewma(np.array(losses_val), span=1.2)
    
    plt.subplot(221)
    plt.plot(np.arange(len(losses_val)) * Training.mod, losses_val, 'r', losses_train, 'b')
    plt.grid()
    plt.legend(['y = loss valid', 'y = loss train'], loc='upper right')

    plt.subplot(222)
    plt.plot(losses_train[-120:])
    plt.grid()

    plt.subplot(223)
    plt.plot(losses_val)
    plt.grid()

    plt.subplot(224)
    plt.plot(losses_val[-60:])
    plt.grid()

    plt.show()
    
    
training_c.set_ploter(my_plot)

In [ ]:
training_c.training(dataDir)

In [ ]:
save_net(net['last'], 'new_cnn2.3v_1.net', dataDir)

In [ ]:
net.learning_rate.get_value()

In [ ]:
net.learning_rate.set_value(0.0006)